# Example Usage of the PanelImputer class

Prepare the environment.

In [1]:
import pandas as pd
# load the class
from panel_imputer import PanelImputer

Let's read some typical political science data like freedom house scores. After a bit of preprocessing we get a nice panel.

(This example requires openpyxl to read the Excel file.)

In [2]:
# Freedomhouse Data: download from https://freedomhouse.org/sites/default/files/2024-02/Aggregate_Category_and_Subcategory_Scores_FIW_2003-2024.xlsx
fp_fh = "Aggregate_Category_and_Subcategory_Scores_FIW_2003-2024.xlsx"
# read data
df_free = pd.read_excel(fp_fh, sheet_name='FIW06-24')

rename_cols = {
    'Country/Territory': 'country',
    'Edition': 'year',  # needs to be -1 in Data
    'PR Rating': 'fh_political_rights_rating',
    'CL Rating': 'fh_civil_liberties_rating',
    'A': 'fh_electoral_process',
    'B': 'fh_pol_pluralism_participation',
    'C': 'fh_gov_functioning',
    'PR': 'fh_political_rights_score',  # 40 max
    'D': 'fh_freedom_expression',
    'E': 'fh_assoc_org_rights',
    'F': 'fh_rule_of_law',
    'G': 'fh_individual_rights',
    'CL': 'fh_civil_liberties_score',  # 60 max
    'Total': 'fh_score'
}
df_free = df_free.dropna(axis=1, how='all').rename(columns=rename_cols)
df_free = df_free.drop(columns=[col for col in df_free.columns if col not in rename_cols.values()])

# some processing necessary:
df_free['year'] = df_free.year - 1  # FH Edition is given in the data, which refers to observations from the previous year
df_free = df_free.sort_values(['country', 'year'])
df_free['time'] = df_free.year.apply(lambda x: pd.Timestamp(x, 12, 31))

df_free

,country,year,fh_political_rights_rating,fh_civil_liberties_rating,fh_electoral_process,fh_pol_pluralism_participation,fh_gov_functioning,fh_political_rights_score,fh_freedom_expression,fh_assoc_org_rights,fh_rule_of_law,fh_individual_rights,fh_civil_liberties_score,fh_score,time
3763,Abkhazia,2005,5,5,5,5,3,13,8,4,4,5,21,34,2005-12-31
3556,Abkhazia,2006,5,5,5,5,3,13,8,4,4,5,21,34,2006-12-31
3349,Abkhazia,2007,5,5,5,6,4,15,8,4,4,5,21,36,2007-12-31
3140,Abkhazia,2008,5,5,5,5,4,14,8,4,4,5,21,35,2008-12-31
2932,Abkhazia,2009,5,5,5,5,4,14,7,5,4,5,21,35,2009-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,Zimbabwe,2019,5,5,3,6,3,12,7,3,2,5,17,29,2019-12-31
839,Zimbabwe,2020,6,5,3,5,3,11,7,3,2,5,17,28,2020-12-31
629,Zimbabwe,2021,6,5,3,5,3,11,7,3,2,5,17,28,2021-12-31
419,Zimbabwe,2022,6,5,3,5,3,11,7,3,2,5,17,28,2022-12-31


Let's say we want to do a monthly model using freedom house data. First, create a monthly data structure while assigning the observation to the last month of each year.

Since the imputer works based on location and time information in the index, we need to create an appropriate multiindex in the process.

In [3]:
df_list = []
for country in df_free.country.unique():
    df_empty = pd.DataFrame(index=pd.date_range(pd.Timestamp(2005,1,31), pd.Timestamp(2021,12,31), freq='ME', name='time')) # depending on the pandas version, freq="ME" or freq="M"
    # we run into some issues with inconsistent country naming over time, so we just don't use those for this example :)
    if len(df_free[df_free.country==country]) == 19: # in this version, there should be 19 entries per country for the full time series
        df_country = df_empty.merge(df_free[df_free.country==country].set_index('time'), how='left', left_index=True, right_index=True)
        df_country.country = df_country.country.bfill().ffill()
        df_list.append(df_country)
df = pd.concat(df_list).drop(columns='year').set_index('country', append=True)
df

,,fh_political_rights_rating,fh_civil_liberties_rating,fh_electoral_process,fh_pol_pluralism_participation,fh_gov_functioning,fh_political_rights_score,fh_freedom_expression,fh_assoc_org_rights,fh_rule_of_law,fh_individual_rights,fh_civil_liberties_score,fh_score
time,country,,,,,,,,,,,,
2005-01-31,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02-28,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-03-31,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04-30,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-05-31,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-30,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-31,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Imputation
Now we have our dataframe with a location/time multiindex, lets see how the imputer works. First: simply backfilling the values for the whole year. In case of NaNs as most recent values, which we don't have in this example, the imputer uses the last seen value.

In [4]:
imp_bfill = PanelImputer(
    location_index='country',
    time_index='time',
    imputation_method='bfill'
)

df_imputed_bfill = imp_bfill.fit_transform(df)
df_imputed_bfill

,,fh_political_rights_rating,fh_civil_liberties_rating,fh_electoral_process,fh_pol_pluralism_participation,fh_gov_functioning,fh_political_rights_score,fh_freedom_expression,fh_assoc_org_rights,fh_rule_of_law,fh_individual_rights,fh_civil_liberties_score,fh_score
time,country,,,,,,,,,,,,
2005-01-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-02-28,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-03-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-04-30,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-05-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-09-30,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-10-31,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0


Second: The class also implements an interpolation option. For this, we can pass arguments to define the interpolation method (see the pandas.interpolate documentation for this) and the behavior at the tails, whether to extrapolate or fill the last seen value. The tail behavior can be passed as string, which is applied to both ends, or as tuple/list of length 2, to treat ends separately.

(Scroll down to Afghanistan to see the behavior in the output, there is no change in the Abkhazia data between 2005 and 2006 so interpolation results in the same values as filling except possibly for floating point precision errors.)

In [5]:
imp_interp = PanelImputer(
    location_index='country',
    time_index='time',
    imputation_method='interpolate',
    interp_method='slinear',
    tail_behavior= 'extrapolate', # alternative: e.g. ['extrapolate', 'fill']
)

df_imputed_interp = imp_interp.fit_transform(df)
df_imputed_interp

,,fh_political_rights_rating,fh_civil_liberties_rating,fh_electoral_process,fh_pol_pluralism_participation,fh_gov_functioning,fh_political_rights_score,fh_freedom_expression,fh_assoc_org_rights,fh_rule_of_law,fh_individual_rights,fh_civil_liberties_score,fh_score
time,country,,,,,,,,,,,,
2005-01-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-02-28,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-03-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-04-30,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-05-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-09-30,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-10-31,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0


### Parallelization
Especially with larger dataframes, imputation can become quite computationally costly. PanelImputer therefore also implements a simple parallelization option using joblib's Parallel.

Lets create a daily version for this dataset (however nonsensical this may be), which makes this over a million rows, and compare how long squential interpolation takes here compared to a parallelized version.

In [6]:
df_list = []
for country in df_free.country.unique():
    df_empty = pd.DataFrame(index=pd.date_range(pd.Timestamp(2005,1,31), pd.Timestamp(2021,12,31), freq='D', name='time')) # depending on the pandas version, possibly freq="ME"
    # we run into some issues with inconsistent country naming over time, so we just don't use those for this example :)
    if len(df_free[df_free.country==country]) == 19: # in this version, there should be 19 entries per country for the full time series
        df_country = df_empty.merge(df_free[df_free.country==country].set_index('time'), how='left', left_index=True, right_index=True)
        df_country.country = df_country.country.bfill().ffill()
        df_list.append(df_country)
df_daily = pd.concat(df_list).drop(columns='year').set_index('country', append=True)

In [7]:
# sequential processing of locations
df_daily_imputed = imp_interp.fit_transform(df_daily)

In [8]:
# parallel processing of locations
imp_parallel = PanelImputer(
    location_index='country',
    time_index='time',
    imputation_method='interpolate',
    interp_method='slinear',
    tail_behavior= 'extrapolate',
    parallelize = True,
    parallel_kwargs = {"n_jobs": -1, "verbose": 1}
)

df_daily_imputed_parallel = imp_parallel.fit_transform(df_daily)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  16 | elapsed:    3.2s remaining:   22.4s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    3.8s finished


In [9]:
assert df_daily_imputed.equals(df_daily_imputed_parallel)
df_daily_imputed_parallel

,,fh_political_rights_rating,fh_civil_liberties_rating,fh_electoral_process,fh_pol_pluralism_participation,fh_gov_functioning,fh_political_rights_score,fh_freedom_expression,fh_assoc_org_rights,fh_rule_of_law,fh_individual_rights,fh_civil_liberties_score,fh_score
time,country,,,,,,,,,,,,
2005-01-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-02-01,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-02-02,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-02-03,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-02-04,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-12-28,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-12-29,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0


As we can see, even though parallelization comes with some overhead in terms of spawning new processes, this version is already ~30% faster, with higher gains for even larger datasets.

### All-NA Location Imputation

Combining different datasets with varying coverage can lead to non-imputable locations with filling/interpolation approaches, requiring the need to drop relevant columns or locations to work with models that cannot handle NA values. As an alternative, the Panel Imputer implements a few simple approaches to fill such all-NA locations.

To demonstrate, we set a few countries to NA for separate columns throughout the dataset. As we can see, there are still NA values after imputation with the previous strategy.

In [10]:
import numpy as np

df_na = df.copy()
locs = df_na.index.get_level_values("country").unique()
for i, col in enumerate(df_na.columns):
    df_na.loc[(slice(None), locs[i]), col] = np.nan
    
imp_interp.fit_transform(df_na)

,,fh_political_rights_rating,fh_civil_liberties_rating,fh_electoral_process,fh_pol_pluralism_participation,fh_gov_functioning,fh_political_rights_score,fh_freedom_expression,fh_assoc_org_rights,fh_rule_of_law,fh_individual_rights,fh_civil_liberties_score,fh_score
time,country,,,,,,,,,,,,
2005-01-31,Abkhazia,NaN,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-02-28,Abkhazia,NaN,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-03-31,Abkhazia,NaN,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-04-30,Abkhazia,NaN,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-05-31,Abkhazia,NaN,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-09-30,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-10-31,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0


Setting the nan_loc_policy, we can add an additional step to the imputation, filling the all-NA locations again.

In [11]:
imp_interp_nafill = PanelImputer(
    location_index='country',
    time_index='time',
    imputation_method='interpolate',
    interp_method='slinear',
    tail_behavior= 'extrapolate',
    nan_loc_policy='knnimpute'
)

df_na_filled = imp_interp_nafill.fit_transform(df_na)
print("Any NAs detected in columns?\n")
print(df_na_filled.isna().any())
df_na_filled

Any NAs detected in columns?

fh_political_rights_rating        False
fh_civil_liberties_rating         False
fh_electoral_process              False
fh_pol_pluralism_participation    False
fh_gov_functioning                False
fh_political_rights_score         False
fh_freedom_expression             False
fh_assoc_org_rights               False
fh_rule_of_law                    False
fh_individual_rights              False
fh_civil_liberties_score          False
fh_score                          False
dtype: bool


,,fh_political_rights_rating,fh_civil_liberties_rating,fh_electoral_process,fh_pol_pluralism_participation,fh_gov_functioning,fh_political_rights_score,fh_freedom_expression,fh_assoc_org_rights,fh_rule_of_law,fh_individual_rights,fh_civil_liberties_score,fh_score
time,country,,,,,,,,,,,,
2005-01-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-02-28,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-03-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-04-30,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
2005-05-31,Abkhazia,5.0,5.0,5.0,5.0,3.0,13.0,8.0,4.0,4.0,5.0,21.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-09-30,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0
2021-10-31,Zimbabwe,6.0,5.0,3.0,5.0,3.0,11.0,7.0,3.0,2.0,5.0,17.0,28.0


### Use in Pipelines
Lastly: The TimeSeriesImputer can be used as part of sklearn pipelines, as illustrated in this (nonsensical) example with a random target.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import numpy as np
X = df
y = pd.Series(np.random.choice([0,1], size=len(df)), index=df.index, name='target') # random dummy target

pipe = Pipeline(
    [('impute', imp_interp),
    ('model', RandomForestClassifier())]
)

pipe.fit(X, y)

,steps,"[('impute', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,location_index,'country'
,time_index,'time'
,missing_values,nan
,imputation_method,'interpolate'
,interp_method,'slinear'
,tail_behavior,'extrapolate'
,nan_loc_policy,None


Just for fun, lets check our AUROC performance:

In [13]:
from sklearn.metrics import roc_auc_score
y_hat = pipe.predict_proba(X)[:,1]
roc_auc_score(y, y_hat)

0.9027249632410603